In [19]:
from IPython.display import HTML, display
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
import numpy as np
import scipy.interpolate as interp
import matplotlib.pyplot as plt
import math
from scipy import stats
import glob
from scipy.interpolate import interp1d
import mplcursors
%matplotlib inline
from matplotlib.widgets import Slider, Button, RadioButtons
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import IPython

from Vflat_importfunctions import changing_ML2,  Vflat_recurse

import warnings
warnings.filterwarnings("ignore")
plt.close()

In [4]:
Lbulge  = np.loadtxt('sparc global measurements.csv', delimiter=',', unpack = True, skiprows = 2, usecols = (20)) 
Vflat_allgals = np.empty(175)
sparc_new = {} #disctionary to hold all values
n=0 #used for indexing purposes
name = np.empty(175, dtype = object) #stores name of galaxy
haghi_MLstellar, chan_MLdisk, chan_MLbulge = np.loadtxt('calculating Vflat from other papers.csv', delimiter=',', unpack = True, skiprows = 2, usecols = (2,5,6)) 
paper_names = np.loadtxt('calculating Vflat from other papers.csv', delimiter=',', skiprows = 2, usecols = (0), dtype = 'object', unpack = True)
sparc_vflat = np.empty(len(haghi_MLstellar))
haghi_vflat = np.empty(len(haghi_MLstellar))
chan_vflat = np.empty(len(haghi_MLstellar))

for filename in sorted(glob.glob('dat files for all galaxies/*_rotmod.dat')):
    radius, vobs, errV, vgas, vdisk, vbul, SBdisk, SBbul = np.loadtxt(filename, delimiter='	', skiprows = 3, unpack = True)
   
    filename = filename.replace('_rotmod.dat', '')
    filename = filename.replace('dat files for all galaxies/', '')
    
    name[n] = str(filename)
    sparc_new[str(filename) + '_rad'] = radius
    sparc_new[str(filename) + '_Vobs'] = vobs
    sparc_new[str(filename) + '_Vgas'] = vgas
    sparc_new[str(filename) + '_Vdisk'] = vdisk
    sparc_new[str(filename) + '_Vbul'] = vbul
    sparc_new[str(filename) + '_index'] = n
    vbary = np.sqrt(0.5*vdisk**2 + 0.7*vbul**2 + vgas**2 )
    sparc_new[str(filename) + '_VDM'] = np.sqrt(vobs**2 - vbary**2)
    
    n = n+1
    
    

In [5]:
def changing_ML_each(ML_disknew, ML_bulgenew, gal_name):
    
    vbaryorig = np.sqrt(0.5 * sparc_new[gal_name+"_Vdisk"]**2 + 0.7 * sparc_new[gal_name+"_Vbul"]**2 + sparc_new[gal_name+"_Vgas"]**2)
    vobsorig = np.sqrt(vbaryorig**2 + sparc_new[gal_name+"_VDM"]**2)
    
    vbarynew = np.sqrt(ML_disknew * sparc_new[gal_name+"_Vdisk"]**2 + ML_bulgenew * sparc_new[gal_name+"_Vbul"]**2 + sparc_new[gal_name+"_Vgas"]**2)
    vobsnew = np.sqrt(vbarynew**2 + sparc_new[gal_name+"_VDM"]**2)
    
    
    Vflat_orig = 0.5 * (vobsnew[-1] + vobsnew[-2])
    Vbar_sum = vobsnew[-1] + vobsnew[-2]
    
    if (np.abs(vobsnew[-3] - Vflat_orig)/Vflat_orig <= 0.05):
        return Vflat_recurse(3, Vbar_sum, vobsnew)
    else:
        return 0

In [6]:
def changing_ML_stellar(ML_stellar, gal_name, disk_multiplier):
    
    ML_disknew = ML_stellar
    ML_bulgenew = ML_stellar * disk_multiplier
    
    vbaryorig = np.sqrt(0.5 * sparc_new[gal_name+"_Vdisk"]**2 + 0.7 * sparc_new[gal_name+"_Vbul"]**2 + sparc_new[gal_name+"_Vgas"]**2)
    vobsorig = np.sqrt(vbaryorig**2 + sparc_new[gal_name+"_VDM"]**2)
    
    vbarynew = np.sqrt(ML_disknew * sparc_new[gal_name+"_Vdisk"]**2 + ML_bulgenew * sparc_new[gal_name+"_Vbul"]**2 + sparc_new[gal_name+"_Vgas"]**2)
    vobsnew = np.sqrt(vbarynew**2 + sparc_new[gal_name+"_VDM"]**2)

    Vflat_orig = 0.5 * (vobsnew[-1] + vobsnew[-2])
    Vbar_sum = vobsnew[-1] + vobsnew[-2]
    
    if (np.abs(vobsnew[-3] - Vflat_orig)/Vflat_orig <= 0.05):
        return Vflat_recurse(3, Vbar_sum, vobsnew)
    else:
        return 0

In [7]:
for x in range(len(name)):
    Vflat_allgals[x] = changing_ML_each(.5, .7, name[x])

def rmse(predictions, targets):
        return np.sqrt(((predictions - targets) ** 2).mean())

## What's going on in the two graphs below?

I wanted to compare the Vflat that I get with SPARC to those from two other papers: [Ho Chan (2018)](https://iopscience.iop.org/article/10.3847/1538-4357/aab3e6/meta) and [Haghi(2017)](https://academic.oup.com/mnras/article/477/3/4187/4919623). These papers don't explicitly give Vflat, but they provide means of calculating it:
- Ho Chan gives $\mathrm{\Upsilon_{disk}}$ and $\mathrm{\Upsilon_{bulge}}$ for a number of graphs. I can calculate a rotation curve using those, and solve for Vflat.
- Haghi provides a $\mathrm{\Upsilon_{stellar}}$ for each galaxy, and it is assumed from Lelli (2016) that I can say $\mathrm{\Upsilon_{stellar}=\Upsilon_{disk}}$ and then I can get $\mathrm{\Upsilon_{bulge}}$ by doing $\mathrm{\Upsilon_{bulge}=\Upsilon_{disk}}$ * `disk_mult` (a variable on the slider below). Previously, it was assumed that `mult_factor` = 1.4. Once again, after finding $\mathrm{\Upsilon_{disk}}$ and $\mathrm{\Upsilon_{bulge}}$, I make a rotation curve and find Vflat

So, now I have Vflat for those two papers, but I also want to see whether it's possible to assign a constant $\mathrm{\Upsilon_{disk}}$ and $\mathrm{\Upsilon_{bulge}}$ for all galaxies as Lelli thinks, as opposed to a different quantity for each. To do this, I have `ML_disk` on the slider below to set the $\mathrm{\Upsilon_{disk}}$ that all galaxies supposedly possess. Then, I calculate $\mathrm{\Upsilon_{bulge}}$ by multiplying `ML_disk` * `disk_mult` (the latter term is explained above) and once again go through the process of making a rotation curve and finding a Vflat. These values are labeled as "my Vflat".

## How do I calculate new Vobs curves?

Since Vflat is derived from the observed rotation curve, I needed it to change every time I altered the M/L ratios. To do this, I first had to take a step back and calculate the proposed velocity due to dark matter. Since I already had observed and baryonic rotation curves of the galaxies under the origincal conditions of $\mathrm{\Upsilon_{disk} = 0.5, \Upsilon_{bulge}=0.7}$, I used the equation $\mathrm{V_{obs}^2 = V_{bary}^2 + V_{DM}^2}$ to create a dark matter rotation curve. I decided that values in $V_{DM}$ wouldn't change with varying M/L ratios, which allows me to only have to solve for $V_{DM}$ each time I did so. Then, whenver I wanted to calculate a new rotation curve for different M/L, I would use the same equation where
$$ \mathrm{V_{bary} = \sqrt{\Upsilon_{disk,new}|V_{disk}|^2 + \Upsilon_{bulge,new}|V_{bulge}|^2 + |V_{gas}|^2}}$$


In [8]:
def changing_sparc_vflat(ML_disk, disk_mult):
    for x in range(len(haghi_MLstellar)):
        sparc_vflat[x] = changing_ML_each(ML_disk, ML_disk * disk_mult, paper_names[x])
        haghi_vflat[x] = changing_ML_stellar(haghi_MLstellar[x], paper_names[x], disk_mult)
        chan_vflat[x]  = changing_ML_each(chan_MLdisk[x], chan_MLbulge[x], paper_names[x])    
    
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))
    nonzero = np.where((sparc_vflat > 0) & (chan_vflat > 0)  & (haghi_vflat > 0))
    
    ax1.scatter(np.log10(sparc_vflat[nonzero]), sparc_vflat[nonzero]/chan_vflat[nonzero])
    ax1.set_xlabel('log(my Vflat) [km/s]')
    ax1.set_ylabel(r"$\mathrm{\frac{my \ Vflat}{Ho\ Chan's \ Vflat}}$", rotation = 90, fontsize = 15, horizontalalignment = 'right')
    ax1.set_title('comparing to Ho Chan')
    ax1.set_ylim(.7,1.8)
    
    coeff,cov=np.polyfit(np.log10(sparc_vflat[nonzero]), sparc_vflat[nonzero]/chan_vflat[nonzero],1,cov=True)
    coeff_err = np.sqrt(np.diag(cov))
    xfit=np.linspace(min(np.log10(sparc_vflat[nonzero])),max(np.log10(sparc_vflat[nonzero])),1000)
    polynomial=np.poly1d(coeff)
    ax1.plot(xfit,polynomial(xfit),color='green')
    rms = rmse(sparc_vflat[nonzero]/chan_vflat[nonzero], coeff[0] * np.log10(sparc_vflat[nonzero]) + coeff[1])
    print('    Left graph:  slope = {:.6f}. scatter = {:.3f}'.format(coeff[0],rms))    
    
    ax2.scatter(np.log10(sparc_vflat[nonzero]), sparc_vflat[nonzero]/haghi_vflat[nonzero])
    ax2.set_xlabel('log(my Vflat) [km/s]')
    ax2.set_ylabel(r"$\mathrm{\frac{my \ Vflat}{Haghi's \ \ Vflat}}$", rotation = 90, fontsize = 15, horizontalalignment = 'center')
    ax2.yaxis.set_label_coords(-.07, 0.5)
    ax2.set_title('comparing to Haghi')
    ax2.set_ylim(.7,2)
     
    coeff,cov=np.polyfit(np.log10(sparc_vflat[nonzero]), sparc_vflat[nonzero]/haghi_vflat[nonzero],1,cov=True)
    coeff_err = np.sqrt(np.diag(cov))
    xfit=np.linspace(min(np.log10(sparc_vflat[nonzero])),max(np.log10(sparc_vflat[nonzero])),1000)
    polynomial=np.poly1d(coeff)
    ax2.plot(xfit,polynomial(xfit),color='green')
    rms = rmse(sparc_vflat[nonzero]/haghi_vflat[nonzero], coeff[0] * np.log10(sparc_vflat[nonzero]) + coeff[1])
    print('    Right graph: slope = {:.6f}. scatter = {:.3f}'.format(coeff[0],rms))
    
interact(changing_sparc_vflat, ML_disk=widgets.FloatSlider(value=0.5, min=0.01, max=3,step=0.01, continuous_update=False),
        disk_mult=widgets.FloatSlider(value=1.4, min=0.01, max=5,step=0.01, continuous_update=False)) 


interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='ML_disk', max=3.0, min=0.01â€¦

<function __main__.changing_sparc_vflat(ML_disk, disk_mult)>